In [18]:
import glob

wav_files = sorted(glob.glob("../Data/MusicNet_Dataset/musicnet/musicnet/train_data/*.wav"))
csv_files = sorted(glob.glob("../Data/MusicNet_Dataset/musicnet/musicnet/train_labels/*.csv"))


In [19]:
import time
import torch.optim as optim
import torch.utils.data as data
from Model.CNN_3L_pro import *
import os

# start_file = "../Data/MusicNet_Dataset/musicnet/musicnet/train_data/2443.wav"
# start_index = wav_files.index(start_file) if start_file in wav_files else 0
# 
# wav_files = wav_files[start_index:]
# csv_files = csv_files[start_index:]
# 
# print(f"Bắt đầu train tiếp từ file: {wav_files[0]}")

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

# Khởi tạo mô hình, hàm mất mát và optimizer
model = CNN_Pro().to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

if os.path.exists("CNN_Pro2.pth"):       
    checkpoint = torch.load("CNN_Pro2.pth", map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

# Huấn luyện từng file một
for wav_path, csv_path in zip(wav_files, csv_files):
    print(time.strftime("%H:%M:%S")+f"\nfile : {wav_path}")
    # Load dữ liệu
    X_train, y_train = load_wav_csv(wav_path, csv_path)
    # Tạo DataLoader
    dataset = MusicDataset(X_train, y_train)
    train_loader = data.DataLoader(dataset, batch_size=64, shuffle=False)

    for epoch in range(20):  # Giảm số epoch cho từng file
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device).float()
            
            optimizer.zero_grad()
            outputs = model(inputs)
             # BCEWithLogitsLoss yêu cầu labels dạng float
            loss = criterion(outputs, labels)  
            loss.backward()
            optimizer.step()
        
            running_loss += loss.item()
            # Chuyển đầu ra thành nhị phân (multi-label classification)
            predicted = (outputs > 0.5).float()
        
            correct += (predicted == labels).sum().item() / 128  # Chia 128 để chuẩn hóa
            total += labels.size(0)

        if epoch >= 19:
            print(f"Epoch {epoch+1}/20, Loss: {running_loss/len(train_loader):.4f}, "
              f"Accuracy: {100 * correct / total:.2f}%")

    torch.save({
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict()
    }, "CNN_Pro2.pth")
    print(f"Updated checkpoint sau file {wav_path}")

print("done training !")

21:49:56
file : ../Data/MusicNet_Dataset/musicnet/musicnet/train_data/1727.wav
Tổng số frame: 38507
Epoch 20/20, Loss: 0.1087, Accuracy: 96.41%
Updated checkpoint sau file ../Data/MusicNet_Dataset/musicnet/musicnet/train_data/1727.wav
21:51:54
file : ../Data/MusicNet_Dataset/musicnet/musicnet/train_data/1728.wav
Tổng số frame: 21632
Epoch 20/20, Loss: 0.0625, Accuracy: 97.46%
Updated checkpoint sau file ../Data/MusicNet_Dataset/musicnet/musicnet/train_data/1728.wav
21:52:57
file : ../Data/MusicNet_Dataset/musicnet/musicnet/train_data/1729.wav
Tổng số frame: 38287
Epoch 20/20, Loss: 0.0873, Accuracy: 96.80%
Updated checkpoint sau file ../Data/MusicNet_Dataset/musicnet/musicnet/train_data/1729.wav
21:54:56
file : ../Data/MusicNet_Dataset/musicnet/musicnet/train_data/1730.wav
Tổng số frame: 31744
Epoch 20/20, Loss: 0.0864, Accuracy: 96.81%
Updated checkpoint sau file ../Data/MusicNet_Dataset/musicnet/musicnet/train_data/1730.wav
21:58:55
file : ../Data/MusicNet_Dataset/musicnet/musicnet/t

In [22]:
import glob
import torch

test_wav_files = sorted(glob.glob("../Data/MusicNet_Dataset/musicnet/musicnet/test_data/*.wav"))
test_csv_files = sorted(glob.glob("../Data/MusicNet_Dataset/musicnet/musicnet/test_labels/*.csv"))

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

model = CNN_Pro().to(device)
checkpoint = torch.load("../Trained/CNN_Pro2.pth", map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()  # Đưa model vào chế độ đánh giá

import numpy as np
import torch.utils.data as data

total_correct = 0
total_samples = 0



for wav_path, csv_path in zip(test_wav_files, test_csv_files):
    print(f"\nĐang test file: {wav_path}")

    X_test, y_test = load_wav_csv(wav_path, csv_path)

    test_dataset = MusicDataset(X_test, y_test)
    test_loader = data.DataLoader(test_dataset, batch_size=64, shuffle=False)

    correct = 0
    total = 0

    with torch.no_grad():  # Không tính gradient khi test
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device).float()

            outputs = model(inputs)
            predicted = (outputs > 0.5).float()  # Chuyển thành 0 hoặc 1
            
            correct += (predicted == labels).sum().item() / 128  # Chia 128 để chuẩn hóa
            total += labels.size(0)

    accuracy = 100 * correct / total
    print(f"Accuracy: {accuracy:.2f}%")

    total_correct += correct
    total_samples += total

print(f"\n🔥 Accuracy chung trên tập test: {100 * total_correct / total_samples:.2f}%")

/var/folders/fr/m00k3ffd001fy8ptw38pn78h0000gn/T/ipykernel_21446/4174036012.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load("../Trained/CNN_Pro2


Đang test file: ../Data/MusicNet_Dataset/musicnet/musicnet/test_data/1759.wav
Tổng số frame: 16768
Accuracy: 98.05%

Đang test file: ../Data/MusicNet_Dataset/musicnet/musicnet/test_data/1819.wav
Tổng số frame: 15287
Accuracy: 97.76%

Đang test file: ../Data/MusicNet_Dataset/musicnet/musicnet/test_data/2106.wav
Tổng số frame: 19523
Accuracy: 97.99%

Đang test file: ../Data/MusicNet_Dataset/musicnet/musicnet/test_data/2191.wav
Tổng số frame: 8859
Accuracy: 99.21%

Đang test file: ../Data/MusicNet_Dataset/musicnet/musicnet/test_data/2298.wav
Tổng số frame: 13224
Accuracy: 99.26%

Đang test file: ../Data/MusicNet_Dataset/musicnet/musicnet/test_data/2303.wav
Tổng số frame: 7981
Accuracy: 98.27%

Đang test file: ../Data/MusicNet_Dataset/musicnet/musicnet/test_data/2382.wav
Tổng số frame: 10164
Accuracy: 98.37%

Đang test file: ../Data/MusicNet_Dataset/musicnet/musicnet/test_data/2416.wav
Tổng số frame: 11984
Accuracy: 98.51%

Đang test file: ../Data/MusicNet_Dataset/musicnet/musicnet/test_d

In [23]:
import torch
import numpy as np
import librosa
import librosa.display

def midi_to_note(midi_note):
    return librosa.midi_to_note(midi_note)

def predict_notes(wav_path, model, device="cpu", sr=44100, hop_length=512, window_size=128, step=21):
    print(f"\n🔍 Đang dự đoán file: {wav_path}")

    # Load file WAV
    y, sr = librosa.load(wav_path, sr=sr)
    
    # Tạo Mel spectrogram
    mel_spec = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128, hop_length=hop_length)
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)

    # Chia thành các cửa sổ (windows)
    X_windows = []
    timestamps = []
    frame_duration = step * hop_length / sr  # Khoảng thời gian của mỗi frame

    for i in range(0, mel_spec_db.shape[1] - window_size, step):
        X_windows.append(mel_spec_db[:, i:i + window_size])
        timestamps.append(i * hop_length / sr)  # Thời gian tính bằng giây
    
    # Chuyển thành tensor
    X_windows = torch.tensor(np.array(X_windows), dtype=torch.float32).unsqueeze(1).to(device)

    batch_size = 32  # Giảm batch size để tránh hết RAM
    num_batches = len(X_windows) // batch_size + 1
    predictions = []
    
    with torch.no_grad():
        for i in range(num_batches):
            batch = X_windows[i * batch_size:(i + 1) * batch_size]
            if batch.shape[0] == 0:
                continue  # Bỏ qua batch rỗng cuối cùng
            outputs = model(batch)
            predictions.append((outputs > 0.5).float().cpu().numpy())
    
    predictions = np.concatenate(predictions, axis=0)  # Ghép lại thành một mảng
    # Xử lý duration của nốt nhạc
    active_notes = {}  # Lưu trạng thái nốt đang được phát
    note_events = []  # Lưu kết quả cuối cùng

    for i, (time, pred) in enumerate(zip(timestamps, predictions)):
        notes = np.where(pred == 1)[0]  # Lấy danh sách các nốt có giá trị 1

        new_active_notes = set(notes)  # Chuyển sang tập hợp để dễ kiểm tra
        
        # Kiểm tra nốt nào vẫn tiếp tục hoặc mới bắt đầu
        for note in new_active_notes:
            if note not in active_notes:
                active_notes[note] = {"start": time, "duration": frame_duration}
            else:
                active_notes[note]["duration"] += frame_duration

        # Kiểm tra nốt nào đã kết thúc
        ended_notes = set(active_notes.keys()) - new_active_notes
        for note in ended_notes:
            note_events.append({
                "note": midi_to_note(note),  # Chuyển đổi MIDI thành tên nốt
                "duration": active_notes[note]["duration"]
            })
            del active_notes[note]

    # Ghi nhận các nốt còn sót lại (kết thúc ở frame cuối)
    for note, info in active_notes.items():
        note_events.append({
            "note": midi_to_note(note),  # Chuyển đổi MIDI thành tên nốt
            "duration": info["duration"]
        })

    return note_events  # Trả về danh sách nốt nhạc

import gc
import torch

gc.collect()
torch.mps.empty_cache()  # Dọn bộ nhớ GPU trên Mac M1/M2
# Ví dụ sử dụng:
wav_path = "../Data/MusicNet_Dataset/musicnet/musicnet/test_data/1819.wav"
notes = predict_notes(wav_path, model, device=device)

print(notes)


🔍 Đang dự đoán file: ../Data/MusicNet_Dataset/musicnet/musicnet/test_data/1819.wav
[{'note': 'A♯4', 'duration': 0.2438095238095238}, {'note': 'A♯4', 'duration': 0.4876190476190476}, {'note': 'D4', 'duration': 0.2438095238095238}, {'note': 'F5', 'duration': 0.7314285714285714}, {'note': 'A♯4', 'duration': 0.4876190476190476}, {'note': 'G5', 'duration': 0.4876190476190476}, {'note': 'D♯5', 'duration': 0.7314285714285714}, {'note': 'D♯4', 'duration': 0.2438095238095238}, {'note': 'G5', 'duration': 0.2438095238095238}, {'note': 'C5', 'duration': 0.4876190476190476}, {'note': 'D♯5', 'duration': 0.2438095238095238}, {'note': 'D5', 'duration': 0.2438095238095238}, {'note': 'F5', 'duration': 0.2438095238095238}, {'note': 'A♯4', 'duration': 0.2438095238095238}, {'note': 'C5', 'duration': 0.9752380952380952}, {'note': 'F5', 'duration': 0.4876190476190476}, {'note': 'A♯4', 'duration': 0.4876190476190476}, {'note': 'A♯4', 'duration': 0.2438095238095238}, {'note': 'A♯4', 'duration': 0.487619047619

In [ ]:
from music21 import stream, note, midi

def export_to_musicxml(note_list, output_file="output.musicxml"):
    s = stream.Stream()
    
    for pitch, duration in note_list:
        n = note.Note(pitch)
        n.quarterLength = duration  # Đặt độ dài nốt theo đơn vị quarter
        s.append(n)
    
    s.write('musicxml', fp=output_file)
    print(f"Xuất thành công: {output_file}")

# Ví dụ danh sách nốt (nốt, trường độ)
notes = [("C4", 1.0), ("D4", 0.5), ("E4", 0.5), ("F4", 1.0), ("G4", 2.0)]

# Xuất file MusicXML
export_to_musicxml(notes)

In [ ]:
import torch
import numpy as np
import librosa

def predict_notes(wav_path, model, device="cpu", sr=44100, hop_length=512, window_size=128, step=21):
    print(f"\n🔍 Đang dự đoán file: {wav_path}")

    y, sr = librosa.load(wav_path, sr=sr)
    
    mel_spec = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128, hop_length=hop_length)
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)

    # Chia thành các cửa sổ (windows)
    X_windows = []
    timestamps = []
    frame_duration = step * hop_length / sr  # Khoảng thời gian của mỗi frame

    for i in range(0, mel_spec_db.shape[1] - window_size, step):
        X_windows.append(mel_spec_db[:, i:i + window_size])
        timestamps.append(i * hop_length / sr)  # Thời gian tính bằng giây
    
    # Chuyển thành tensor
    X_windows = torch.tensor(np.array(X_windows), dtype=torch.float32).unsqueeze(1).to(device)

    model.eval()
    
    batch_size = 32  # Có thể giảm xuống 16 nếu vẫn hết RAM
    num_batches = len(X_windows) // batch_size + 1
    predictions = []
    
    with torch.no_grad():
        for i in range(num_batches):
            batch = X_windows[i * batch_size:(i + 1) * batch_size]
            if batch.shape[0] == 0:
                continue
            outputs = model(batch)
            predictions.append((outputs > 0.5).float().cpu().numpy())
    
    predictions = np.concatenate(predictions, axis=0)

    # Xử lý duration của nốt nhạc
    active_notes = {}  # Lưu trạng thái nốt đang được phát
    note_events = []  # Lưu kết quả cuối cùng

    for i, (time, pred) in enumerate(zip(timestamps, predictions)):
        notes = np.where(pred == 1)[0]  # Lấy danh sách các nốt có giá trị 1

        new_active_notes = set(notes)  # Chuyển sang tập hợp để dễ kiểm tra
        
        # Kiểm tra nốt nào vẫn tiếp tục hoặc mới bắt đầu
        for note in new_active_notes:
            if note not in active_notes:
                active_notes[note] = {"start": time, "duration": frame_duration}
            else:
                active_notes[note]["duration"] += frame_duration

        # Kiểm tra nốt nào đã kết thúc
        ended_notes = set(active_notes.keys()) - new_active_notes
        for note in ended_notes:
            note_events.append({
                "note": note,
                "start": active_notes[note]["start"],
                "end": active_notes[note]["start"] + active_notes[note]["duration"],
                "duration": active_notes[note]["duration"]
            })
            del active_notes[note]

    # Ghi nhận các nốt còn sót lại (kết thúc ở frame cuối)
    for note, info in active_notes.items():
        note_events.append({
            "note": note,
            "start": info["start"],
            "end": info["start"] + info["duration"],
            "duration": info["duration"]
        })

    # Hiển thị kết quả
    print("\n🎼 Kết quả dự đoán:")
    for idx, event in enumerate(sorted(note_events, key=lambda x: x["start"])):
        print(f"{idx+1}. 🎵 Note {event['note']} - Start: {event['start']:.2f}s, End: {event['end']:.2f}s, Duration: {event['duration']:.2f}s")
import gc
import torch

gc.collect()
torch.mps.empty_cache() 
# Dự đoán cho một file WAV
wav_path = "../Data/MusicNet_Dataset/musicnet/musicnet/test_data/1819.wav"
predict_notes(wav_path, model, device=device)